# Task 2 - Semantic Search

In this notebook, I build a semantic search algorithm to search for jobs within a descriptions database, that are most closely match the user search criteria. Here, I use the "descriptions" data column of job_postings.csv using the sqlite job_postings.db database created in the sql_schema.ipynb. The search algorithm is based on the "all-MiniLM-L6-v2" model from the `sentence_transformers` library and embeds each descriptions paragraph with a 384 dimentional vector. The model embeds 7066 samples  of descriptions using the `extract_text_and_jobid` function. The top 10 most semantically similar results are displayed within their respective sections.

## Performance assestment
The model performance was evaluated on all 7066 job descriptions within the job_postings.csv that it was trained on,
No testing set was used.

The following 3 queries were used to test the model performance

1. I'm looking for a job that requires a nursing degree
2. Construction work, civil engineering, architecture
3. Are there any medical research jobs?

In order to asess the performance of the algorithm I simply scanned the results dataset of each query to see how well the algorithm was able to return a relevant result. Often times the model was able to output a result that was relevant to the query, but not in all cases. For example when using the "Construction work, civil engineering, architecture" query, its rank 3 suggestion was much too high for the desciption paragraph to warrant 

`Rank 3:  ['Skills:Category plan development and management large dollar risk purchasesSupplier development and performanceNegotiations and competitive processContract development, management, and administration (may include Construction Specifications Institute)Spend and market analysisPlanning and forecastingContinuous improvement and productivityRelationship Management'] `

Very little is said about Construction work, civil engineering or architecture.. it is possible that this paragraph recived such a high ranking because of the explicit word "construction". 


More rigorously, one could treat this as a classification task and use a ROC curve to evaluate performance. Step 1 would be to label all descriptions with classification tag, giving us a true positive dataset for each class. We could maybe use the 'industry' characteristic in the company_industries.csv as a truth label, and create a standardized question ask the agent, ex. "Retreive any postings related to the "X" industry", and evaluate TPR and FPR metrics to asess model performance. As well as evaluate the model on a labeled testing dataset.

One might, less rigorously be able to count number of times key words from the search string appear in the job descriptions and create a similarity matrix of our own to assess performance. The key phrases in the queries above count be, "nursing degree' "Construction", 'civil engineering", "architecture", and "medical research".



## Methods to improve algorithm
1. Perhaps using a more sophisticated LLM model architecture such as Ollama3 or ChatGPT-4o would provide more powerful sematinic matching. This would allow for the words within the search phrase itsself to inform the model of the true meaning of any homonyms used.
2. Cleaning of the description data might also help model performance, as can be seen below, the description paragraphs contained hidden "\n", "\n\n", ,"\u202", etc. characters that could be preventing the model from identifying key words properly. I notice that the description paragraphs still contain spacing typos such as 'weekSalary'  which the model might interpret as 1 word, potentially affecting model performance.



In [1]:
# Importing important libraries
import chromadb
import os
import sys
import sqlite3
import numpy as np
from sentence_transformers import SentenceTransformer

In [2]:
# Ensuring compatability with ChromaDB
sys.version

'3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]'

# Prepping the sql database for parsing

Helpful link for understanding vector DBs
https://dev.to/aditya47x/chromadb-for-the-sql-mind-1b00

In [3]:
# Establishing a connection to an exisiting database used in sql task
conn = sqlite3.connect("job_postings.db")

# Queries for extracting only the 'description', amd jobID column
query = "SELECT description FROM fact_job_postings"
job_id_query = "SELECT job_id FROM fact_job_postings"

# Extracting the text and jobID from the SQL database

In [4]:
# Creating a function that will allow me to extract a subset of the original dataset for searches (allows for iterating, saves on computation)
def extract_text_and_jobid(connection, description_query, job_id_query , n_samples = -1):
    '''
    A function that will extract a given number `n_samples` of descriptions from the given sql database
    in `connection` and uses that for the for the search pool.
    '''

    # Will be extracting all text and job_ids (Computationally expensive)
    if n_samples==-1:
        # Extracting the text from the descriptions column
        texts   = [row[0] for row in connection.execute(description_query).fetchall()]
        # Extracting the job ID could use this later to look up salary etc. , ids must be strings
        job_ids = [str(id_row[0]) for id_row in connection.execute(job_id_query).fetchall()]
    else:
        texts   = [row[0] for row in conn.execute(description_query).fetchall()[:n_samples]]
        job_ids = [str(id_row[0]) for id_row in connection.execute(job_id_query).fetchall()[:n_samples]]
        
    return texts, job_ids

In [5]:
# Do the actal extraction, this time for all data samples
texts, job_ids = extract_text_and_jobid(conn, query, job_id_query, n_samples = -1)

# Check that all were indeed extracted
print(len(texts), len(job_ids))

7066 7066


In [6]:
# Looks like there are '\n' new line characters in the descriptions, should maybe remove these since they could interfere with searches..
print(job_ids[100])
texts[100]

3884437755


'Our beautiful practice in Niagara Falls is looking for a General Dentist! The ideal candidate will work along side our senior doctor for a few years and then purchase the practice. This is a great opportunity as the building is owned, not leased! Our practice is very well established in the community with a steady patient flow and high productivity. It is up date with the latest technology and the tools for the right candidate to be successful! \nPerks:Working for a doctor-owned practiceComplete clinical autonomyMentorshipHealth InsuranceCE AllowanceJob Duties:Examine, diagnose, prescribe, and carry out services and treatment plans.Collaborate with other providers, as well as all other clinical and non-clinical personnel as necessary.Uphold the policy, protocol, and procedures which are in compliance with the most current accepted professional standards.Improving clinical skills and acumen through participation in continuing education and training opportunities (continuing education s

In [7]:
# Replacing '\n' new line characters in the descriptions, since they could interfere with searches..
texts = [row.replace("\n", " ") for row in texts]

In [8]:
# Lets check that I am extracting the descriptions correctly, yes, looks like the \n characters were removed
print(job_ids[100])
texts[100]

3884437755


'Our beautiful practice in Niagara Falls is looking for a General Dentist! The ideal candidate will work along side our senior doctor for a few years and then purchase the practice. This is a great opportunity as the building is owned, not leased! Our practice is very well established in the community with a steady patient flow and high productivity. It is up date with the latest technology and the tools for the right candidate to be successful!  Perks:Working for a doctor-owned practiceComplete clinical autonomyMentorshipHealth InsuranceCE AllowanceJob Duties:Examine, diagnose, prescribe, and carry out services and treatment plans.Collaborate with other providers, as well as all other clinical and non-clinical personnel as necessary.Uphold the policy, protocol, and procedures which are in compliance with the most current accepted professional standards.Improving clinical skills and acumen through participation in continuing education and training opportunities (continuing education su

# 2.1 Pick vector index and embedding providers
Here, the easiest embedding provider was SentenceTransformer, using the default model, as suggested

In [9]:
# We want a database that will be saved for later use (persistent)
chroma_client = chromadb.PersistentClient(path="/Users/chiral/git_projects/data_and_ai_intern_2025/chroma_dbs/job_descriptions_embeddings")

embedding_path = "/Users/chiral/git_projects/data_and_ai_intern_2025/chroma_dbs/job_embeddings.npy"

# Load model embeddings if they exist, if not make them (computationally expensive)
if os.path.isfile(embedding_path):
    # Load up embeddings to save on time
    embeddings = np.load(embedding_path)
else:
    # Manually creating embeddings to keep track of them: https://www.sbert.net/docs/quickstart.html
    model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = model.encode(texts, show_progress_bar=True)
    np.save(embedding_path, embeddings)


Key Request: After embedding all of the descriptions, display the dimension of one of the embedding vectors in your index.

In [10]:
embeddings.shape
# embeddings[0].shape

(7066, 384)

There are 7066 data samples with vector embeddings of dimension 384, intrinsic to model?

# 2.2 Create a job description index

In [11]:
collection = chroma_client.get_or_create_collection(name="job_descriptions")

In [12]:
# Chromadb does not allow for adding to collection with a batch size greater than 5461, must do multiple batch statements apparently
def add_in_batch(collection, text, ids, embeddings):
    batch_size=1000
    for i in range(0, len(text), batch_size):  # Slice list in steps of n
        # Creating a collection to search through
        collection.add(
            documents = texts[i:i + batch_size],  # Each one of these will get an associated vector
            ids = job_ids[i:i + batch_size],      # Lets use the job_id as the ids since these are unique, and can be used to lookup more information form the fact table later 
            embeddings = embeddings[i:i + batch_size] # The mappings of the documents to vectors
        )

In [13]:
# Execute the batching function
add_in_batch(collection, texts, job_ids, embeddings)

# 2.3 Testing my semantic search workflow
Here I will be testing the search algorithm to see if it gives results that make sense, does it match properly?
In general, yes

## Search expression: "I'm looking for a job that requires a nursing degree"

In [14]:
# Performing a text search of the above phrase
results_nurse_deg = collection.query(
    query_texts=["I'm looking for a job that requires a nursing degree"],
    n_results=10 # how many results to return
)

In [15]:
# Listing the kinds of information that are available in the results set
results_nurse_deg.keys()

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances'])

In [16]:
# These are the top 10 semantically close descriptions to my question
print(len(results_nurse_deg['ids'][0]), results_nurse_deg['ids'])

10 [['3904500691', '3904099662', '3903429488', '3903811499', '3886453773', '3886454581', '3903431209', '3903875230', '3903426888', '3903432100']]


In [17]:
# Lets print out the description paragraph for the set of results, see if key words appear
for i, job_id in enumerate(results_nurse_deg["ids"][0]):
    print(f"Rank {i+1}: ", collection.get(ids=[job_id])['documents'], "\n")

Rank 1:  ['Employment Type\n\nFull time\n\nShift\n\n12 Hour Night Shift\n\nDescription\n\nPosition Purpose\n\nAt Saint Francis Hospital, our Acute Care Unit plays a crucial role in providing comprehensive medical care to adult patients. As an Acute Care Registered Nurse (RN), you’ll be at the forefront of patient management, ensuring their well-being and contributing to positive outcomes. Our patient population encompasses a wide range of medical diagnoses, including pulmonary, gastrointestinal, renal, endocrine, circulatory, neurological, infectious, dermatological, and autoimmune disorders.\n\nWhat You Will Do\n\nAs an Acute Care RN, your responsibilities will include:\n\nCollaborating with interdisciplinary teams to create and manage nursing care plans.Participating in the evaluation of current and future medical needs of patients.Administering medications and treatments.Monitoring patient conditions and responding to changes promptly.\n\nMinimum Qualifications\n\nEducation: You mus

## Search expression: "Construction work, civil engineering, architecture"

In [18]:
results_civil_engineering = collection.query(
    query_texts=["Construction work, civil engineering, architecture"],
    n_results=10 # how many results to return
)
# These are the top 10 semantically close descriptions to my question
print(len(results_civil_engineering['ids'][0]), results_civil_engineering['ids'])

10 [['3905327539', '3901947403', '3906097585', '3901379478', '3906249896', '3904969069', '3894565520', '3902351610', '3902834659', '3905368795']]


In [19]:
for i, job_id in enumerate(results_civil_engineering["ids"][0]):
    print(f"Rank {i+1}: ",collection.get(ids=[job_id])["documents"],"\n")

Rank 1:  ["The Opportunity \n\n We are currently seeking a Construction Project Engineer for the Transportation-Construction Management team in the Virginia Beach, VA area. \n\nThe Construction Project Engineer has a can-do attitude and is a self-starter; responsible for helping with the firm's Construction Engineering Inspection Services. You will help grow and develop staff and work closely with clients, engineers, and sub-consultants to ensure their needs are met. You will also be involved in business planning, marketing, and fulfillment of the firm’s strategic vision for the construction services program.\n\nAlso operating as a Construction Project Engineer, you must have the credentials and ability to manage Project Engineers and Construction Engineering Inspection (CEI) teams for road and bridge transportation projects. Additional job duties include implementing quality assurance procedures, preparing and submitting estimates, checking engineering calculations, interpreting contr

## Search expression: "Are there any medical research jobs?"

In [20]:
results_med_research = collection.query(
    query_texts=["Are there any medical research jobs?"],
    n_results=10 # how many results to return
)
# These are the top 10 semantically close descriptions to my question
print(len(results_med_research['ids'][0]), results_med_research['ids'])

10 [['3888931781', '3889414768', '3895538611', '3889417420', '3901691799', '3901699042', '3902832708', '3889411979', '3903839479', '3887715991']]


In [21]:
for i, job_id in enumerate(results_med_research["ids"][0]):
    print(f"Rank {i+1}: ",collection.get(ids=[job_id])["documents"],"\n")

Rank 1:  ['Description:\n\nJohnson and Johnson is recruiting for a Senior Analyst, Clinical Trial Learning & Development located in Titusville, NJ; Beerse, Belgium; High Wycombe, United Kingdom; Toronto, Ontario Canada; Wuhan, China; Tokyo, Japan; Seoul, South Korea; New South Wales, Australia; Singapore, Singapore; Milan, Italy; Vienna, Austria; Torokbalint, Hungary; Issy Les Moulineaux, France; Neuss, Germany; Madrid, Spain; Warsaw, Poland; Birkerod, Denmark; Espoo, Finland; Breda, Leiden, Netherlands; Barcarena, Portugal; Stockholm Sweden; Zug, Switzerland; Sao Paulo, Brazil; Mexico City, Mexico; or Buenos Aires, Capital Federal, Argentina. Remote work options may be considered on a case-by-case basis and if approved by the Company.\n\nAt Johnson & Johnson, we believe health is everything. Our strength in healthcare innovation empowers us to build a world where complex diseases are prevented, treated, and cured, where treatments are smarter and less invasive, and solutions are perso